In [ ]:
from docx import Document
import re

def reestruturar_e_corrigir_docx(caminho_entrada: str, caminho_saida: str):
    """
    Reestrutura e limpa um arquivo .docx gerado por scraping de webnovels.
    
    - Detecta capítulos a partir de linhas com 'Chapter <número>: <título>'.
    - Adiciona quebra de página entre capítulos.
    - Remove subtítulos duplicados (casos em que o título aparece novamente após o original).
    - Reestrutura parágrafos longos para melhorar a legibilidade.
    
    Parâmetros:
    - caminho_entrada: caminho do arquivo .docx original.
    - caminho_saida: caminho onde o novo arquivo será salvo.
    """
    
    # Regex para detectar títulos válidos de capítulos
    padrao_titulo = re.compile(r"^\s*Chapter\s+\d+([:–-]?\s*.+)?\s*$", re.IGNORECASE)

    doc = Document(caminho_entrada)
    novo = Document()
    novo.core_properties.author = "Light_Editor"

    primeiro = True  # Indica se é o primeiro capítulo

    def split_outside_quotes(texto: str):
        """
        Divide um texto longo em sentenças, sem quebrar dentro de aspas.
        """
        partes = []
        buf = ""
        in_q = False
        i = 0
        L = len(texto)
        while i < L:
            c = texto[i]
            buf += c
            if c == '"':
                in_q = not in_q
            elif not in_q and c in ".!?":
                rest = texto[i+1:]
                if re.match(r'\s+[A-Z]', rest) or not rest.strip():
                    partes.append(buf.strip())
                    buf = ""
            i += 1
        if buf.strip():
            partes.append(buf.strip())
        return partes

    def reestruturar_paragrafo(raw: str):
        """
        Quebra parágrafos longos em partes menores:
        - Mantém parágrafos curtos como estão.
        - Preserva falas entre aspas.
        - Quebra narrativas longas fora das falas.
        """
        raw = raw.strip()
        if len(raw) <= 300:
            return [raw]

        partes = re.split(r'(".*?")', raw)
        resultado = []
        i = 0
        while i < len(partes):
            seg = partes[i].strip()
            if not seg:
                i += 1
                continue
            if seg.startswith('"') and seg.endswith('"'):
                bloco = seg
                if i+1 < len(partes):
                    prox = partes[i+1].strip()
                    if prox:
                        palavras = prox.split()
                        if len(palavras) <= 15:
                            bloco += " " + prox
                            partes[i+1] = ""
                resultado.append(bloco)
                i += 1
                continue
            resultado.extend(split_outside_quotes(seg))
            i += 1
        return resultado

    # === FASE 1: Reconstrução do conteúdo ===
    for par in doc.paragraphs:
        full = par.text.strip()
        if not full:
            continue

        if padrao_titulo.match(full):
            if not primeiro:
                novo.add_page_break()
            primeiro = False
            novo.add_paragraph(full, style='Heading 1')
            novo.add_paragraph()  # linha em branco após o título
            continue

        # Divide possíveis linhas dentro de um mesmo parágrafo original
        for raw in full.split('\n'):
            raw = raw.strip()
            if not raw:
                continue
            for bloco in reestruturar_paragrafo(raw):
                novo.add_paragraph(bloco)

    # === FASE 2: Remoção de subtítulos duplicados ===
    def contem_chapter_e_numero(texto: str) -> bool:
        """
        Verifica se o texto contém 'chapter <número>'.
        """
        return bool(re.search(r"chapter\s+\d+", texto.lower()))

    to_delete = []
    for i in range(len(novo.paragraphs) - 1):
        atual = novo.paragraphs[i].text.strip()
        proximo = novo.paragraphs[i + 1].text.strip()
        if padrao_titulo.match(atual) and contem_chapter_e_numero(proximo):
            to_delete.append(i + 1)

    for idx in reversed(to_delete):
        elm = novo.paragraphs[idx]._element
        elm.getparent().remove(elm)

    # === SALVAMENTO FINAL ===
    novo.save(caminho_saida)






In [6]:
reestruturar_e_corrigir_docx("Divine Emperor of Death 2101-2200.docx", "Divine Emperor of Death 2101-2200_lee.docx")
